In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

2025-06-18 03:00:47.336495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750215647.573318      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750215647.641748      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from IPython.display import display
from tqdm.notebook import tqdm

In [4]:
imdb_dataset = load_dataset("porcupine1829/imdb-reviews-data")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/51.5M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
x_train = imdb_dataset['train']['text']
y_train = imdb_dataset['train']['label']

x_test = imdb_dataset['validation']['text']
y_test = imdb_dataset['validation']['label']

# create new dataset
dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                             'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

In [6]:
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.5001

In [7]:
model_checkpoint = 'distilbert-base-uncased'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)


cuda


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def tokenize_function(examples):
    text = examples["text"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [11]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
# Load metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get predicted class indices
    preds = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"]
    return {"accuracy": accuracy, "f1": f1}


In [14]:
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]
print("Untrained model predictions:")
print("----------------------------")

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    with torch.no_grad():  # optional, avoids storing gradients
        logits = model(inputs).logits

    predictions = torch.argmax(logits, dim=1)

    print(text + " - " + id2label[predictions.item()])

Untrained model predictions:
----------------------------
It was good. - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative


In [15]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [16]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [17]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [18]:
# Load metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get predicted class indices
    preds = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [19]:
training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=16,        # Efficient on P100
    per_device_eval_batch_size=32,         # Larger eval batch is okay
    num_train_epochs=num_epochs,                    # Common starting point
    weight_decay=0.01,
    
    eval_strategy="epoch",           # Or "steps" if dataset is large
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    
    logging_strategy="steps",
    logging_steps=50,
    
    fp16=True,                             # Use mixed precision for speed
    dataloader_num_workers=4,              # Use 4-8 based on Kaggle environment
    disable_tqdm=False,
    report_to="none"                       # Avoid WandB unless needed
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("./my_model")
tokenizer.save_pretrained("./my_model")

/tmp/ipykernel_19/1533351332.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers`

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.293700,0.242351,0.900000,0.896566
2,0.246000,0.224984,0.915400,0.916486
3,0.221200,0.224615,0.913100,0.915737
4,0.231300,0.210928,0.924800,0.924679
5,0.177800,0.227011,0.921700,0.920338
6,0.172600,0.228794,0.925500,0.924785
7,0.190900,0.200715,0.929500,0.930260
8,0.144400,0.233965,0.928300,0.928820
9,0.163800,0.231129,0.930500,0.930031
10,0.110500,0.243250,0.931700,0.931897


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

In [21]:
print("Trained model predictions:")
print("--------------------------")

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    with torch.no_grad():  # optional, avoids storing gradients
        logits = model(inputs).logits

    predictions = torch.argmax(logits, dim=1)

    print(text + " - " + id2label[predictions.item()])

Trained model predictions:
--------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Positive


In [22]:
from IPython.display import FileLink
!cd /kaggle/working && tar -czvf my_outputs.tar.gz .
FileLink('my_outputs.tar.gz')

./
./my_model/
./my_model/tokenizer_config.json
./my_model/training_args.bin
./my_model/tokenizer.json
./my_model/vocab.txt
./my_model/adapter_config.json
./my_model/special_tokens_map.json
./my_model/README.md
./my_model/adapter_model.safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


./distilbert-base-uncased-lora-text-classification/
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/tokenizer_config.json
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/rng_state.pth
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/training_args.bin
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/tokenizer.json
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/scaler.pt
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/optimizer.pt
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/vocab.txt
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/adapter_config.json
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/scheduler.pt
./distilbert-base-uncased-lora-text-classification/checkpoint-25000/special_tokens_map.json
./distilbert-base-uncased-lora-text-clas

/kaggle/working/my_outputs.tar.gz